In [1]:
import numpy as np

In [5]:
from keras.models import Sequential,Model
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D, Deconvolution2D, Cropping2D
from keras.layers import Input, Add, Dropout, Permute, add

In [6]:
def Convblock(channel_dimension, block_no, no_of_convs) :
    Layers = []
    for i in range(no_of_convs) :
        Conv_name = "CONV_"+str(block_no)+"_"+str(i+1)
        Layers.append(Convolution2D(channel_dimension),kernel_size = (3,3),padding = "same",activation = "relu",name = Conv_name)
    Layers.append(MaxPooling2D(pool_size=(2,2),stride = (2,2)))
    return Layers

In [ ]:
def Fcn_16(image_size):
    model = Sequential()
    model.add(Permute(1,2,3),input_shape = (image_size,image_size,3))
    
    
    for l in Convblock(64,1,2) :
        model.add(l)
    
    for l in Convblock(128,2,2):
        model.add(l)
    
    